# Ejercicio de curvas elípticas

Sea $\mathbb{F}_{32} = \mathbb{F}_2[\xi]_{\xi^5 + \xi^2 + 1}$. Cada uno de vosotros, de acuerdo a su número de DNI, dispone de una curva elíptica sobre $\mathbb{F}_{32}$ y un punto base dados en el cuadro 6.1.

1. Calcula mediante el algoritmo de Shank o el 9, $log_Q{\mathcal{O}}$.
2. Para tu curva y tu punto base, genera un par de claves pública/privada para un protocolo ECDH.
3. Cifra el mensaje $(\xi^3 + \xi^2 + 1, \xi^4 + \xi^2) \in \mathbb{F}^2_{32}$ mediante el criptosistema de Menezes-Vanstone.
4. Descifra el mensaje anterior.

Como mi DNI es

In [56]:
dni = 77432071

Tenemos que $\text{dni} \bmod 32$ da 

In [57]:
dni.mod(32)

7

Por lo que mis curvas son:

$$
\begin{aligned}
E(A, B) & = E(\xi + 1, \xi^4 + \xi^3 + \xi + 1) \\ 
Q & = (\xi^3 + \xi^2 + \xi, \xi^4 + \xi^3 + \xi^2 + \xi + 1)
\end{aligned}
$$

In [58]:
F.<xi> = GF(2^5, modulus = GF(2)[x](x^5 + x^2 + 1))
E = EllipticCurve(F, [1, xi + 1, 0, 0, xi^4 + xi^3 + xi + 1])
Q = E.point((xi^3 + xi^2 + xi, xi^4 + xi^3 + xi^2 + xi + 1))

In [59]:
E

Elliptic Curve defined by y^2 + x*y = x^3 + (xi+1)*x^2 + (xi^4+xi^3+xi+1) over Finite Field in xi of size 2^5

In [60]:
Q

(xi^3 + xi^2 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1)

## Apartado 1

Vamos a calcular mediante el algoritmo de Shank $\log_Q{\mathcal{O}} = \log_{(\xi^3 + \xi^2 + \xi, \xi^4 + \xi^3 + \xi^2 + \xi + 1)}{\mathcal{O}}$. 

In [127]:
O = E.point(0)
O

(0 : 1 : 0)

Siguiendo la notación de los apuntes, la base es, en este caso, $b = Q$. 

Para aplicar este algoritmo necesitamos una cota del orden de la curva elíptica. Como nos encontramos en $\mathbb{F}_{32}$, $q = 32$, y por el teorema de Hasse, 

$$
\left \lvert E \right \rvert \le q + 1 + \left \lfloor 2 \sqrt{q} \right \rfloor = 32 + 1 + 11 = 44
$$

In [141]:
q = 32
floor(2 * sqrt(q))

11

Tomamos por tanto $f = \left \lceil \sqrt{44} \right \rceil$

In [142]:
f = ceil(sqrt(
    q + 1 + floor(2 * sqrt(q))
))
f

7

Construimos la tabla. En este caso, cada entrada viene dada por 

$$
[i, iQ], \quad 0 \le i \le f-1
$$

In [143]:
tabla = [[i, i*Q] for i in range(0, f)]
tabla

[[0, (0 : 1 : 0)],
 [1, (xi^3 + xi^2 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1)],
 [2, (xi^4 + 1 : xi^2 + xi : 1)],
 [3, (1 : xi^4 + xi^3 + xi : 1)],
 [4, (xi^2 + xi + 1 : xi^4 + xi^3 + xi^2 + xi + 1 : 1)],
 [5, (xi^3 + xi : xi^4 + xi^2 + 1 : 1)],
 [6, (xi^4 + xi^3 + xi : xi : 1)]]

Debemos calcular ahora $Q^{-f} = -f Q = -7Q$

In [144]:
b_elev_menos_f = -f*Q
b_elev_menos_f

(xi^4 + xi^3 + xi^2 + xi : xi^4 + xi^3 : 1)

Vamos a necesitar una función para comprobar si un elemento está en la tabla. Podemos reaprovechar la de los ejercicios anteriores:

In [145]:
def esta_en_tabla(h, tabla):
    matches = [x for x in tabla if x[1] == h]
    return len(matches) > 0

In [146]:
esta_en_tabla(-6 * Q, tabla)

False

Dado que no lo está, debemos ir construyendo $h_i = h_{i - 1} g^{-f}$. Implementemos una función sencilla que nos diga el índice $j$ de la tabla:

In [147]:
def dime_indice_j(h, tabla):
    matches = [x for x in tabla if x[1] == h]
    
    if len(matches) > 0:
        return(matches[0][0])

Finalmente, calculemos el logaritmo:

In [148]:
h = O

In [149]:
indice_i = 0
indice_j = 0

for i in range(1, len(tabla)):
    h = h + (b_elev_menos_f)
    
    print("h_"+ str(i) + " = " + str(h))
    
    if esta_en_tabla(h, tabla):
        indice_i = i
        indice_j = dime_indice_j(h, tabla)
        print("Sí que está en la tabla (i = " + str(indice_i) + ", j = " + str(indice_j) + ")")
        break

h_1 = (xi^4 + xi^3 + xi^2 + xi : xi^4 + xi^3 : 1)
h_2 = (1 : xi^4 + xi^3 + xi : 1)
Sí que está en la tabla (i = 2, j = 3)


Por tanto, el valor final es 

In [151]:
logaritmo = indice_j + indice_i * f
logaritmo

17

Podemos comprobar que es correcto:

In [152]:
logaritmo*Q

(0 : 1 : 0)

## Apartado 2

Para el algoritmo ECDH, fijamos $E = E(a, b)$ de forma que $\|E\| = hn$, con $n$ primo, $h$ pequeño y $\|Q\| = n$.

In [6]:
E.order()

34

In [7]:
Q.order()

17

Podemos tomar 

In [8]:
n = 17
h = 2

In [9]:
n * h == E.order() and n == Q.order()

True

Ahora Alice debe escoger un número aleatorio $a$ que se encuentre entre $2 \le a \le n-1$, y hacer público $(E, Q, aQ)$

In [11]:
a = ZZ.random_element(2, n)
a

8

Alice va a enviar a Bob $P_A = aQ$

In [18]:
P_A = a*Q
P_A

(xi^3 + xi^2 : xi^2 + xi : 1)

Vamos ahora a generar la clave privada. Para ello, Bob escoge $b$ tal que $2 \le b \le n-1$

In [24]:
b = ZZ.random_element(2, n)
b

Bob calcula $P_B = bQ$ y se lo envía a Alice

In [29]:
P_B = b * Q
P_B

(xi^3 + xi^2 : xi^2 + xi : 1)

Bob finalmente calcula $b(P_A)$

In [33]:
b * P_A

(xi^2 + xi + 1 : xi^4 + xi^3 : 1)

Y por su parte, Alice calcula $a(P_B)$

In [34]:
a * P_B

(xi^2 + xi + 1 : xi^4 + xi^3 : 1)

La clave compartida por lo tanto es $(ab)Q = a(P_B) = b(P_A)$

In [35]:
(a * b) * Q == a * (P_B) == b * (P_A)

True

## Apartado 3

La generación de claves en el criptosistema de Menezes-Vanstone sigue los mismos pasos que en el de ECDH, por lo que mantendremos las anteriores. Sin embargo, para preservar la notación de los apuntes de teoría, haremos

In [36]:
k = b

El mensaje $M = (m_1, m_2)$ es

In [37]:
mensaje = [xi^3 + xi^2 + 1, xi^4 + xi^2]
mensaje

[xi^3 + xi^2 + 1, xi^4 + xi^2]

Podemos partir la clave en dos de la forma $(x_0, y_0) = k(aQ)$:

In [44]:
x_0 = (k*(a*Q))[0]
y_0 = (k*(a*Q))[1]

x_0, y_0

(xi^2 + xi + 1, xi^4 + xi^3)

El mensaje que se va a transmitir es 

$$
\begin{aligned}
E(M) & = E(m_1, m_2) = (kQ, M + k(aQ)) = \\ & = (kQ, x_0m_1, y_0m_2)
\end{aligned}
$$

In [46]:
x_0m_1 = mensaje[0] + (k*(a*Q))[0]
y_0m_2 = mensaje[1] + (k*(a*Q))[1]

print(
      "x_0 m_1 = " + str(x_0m_1) 
    + "\ny_0 m_2 = " + str(y_0m_2)
)

x_0 m_1 = xi^3 + xi
y_0 m_2 = xi^3 + xi^2


## Apartado 4

Para descifrar el mensaje, Alice debe hacer

$$
\begin{aligned}
D(kQ, M + k(aQ)) & = M + k(aQ) - a(kQ) = \\ 
                 & = M
\end{aligned}
$$

In [47]:
descifrado = [
    x_0m_1 - (a*(k*Q))[0], 
    y_0m_2 - (a*(k*Q))[1]
]
descifrado

[xi^3 + xi^2 + 1, xi^4 + xi^2]

In [48]:
descifrado == mensaje

True